In [2]:
#Data input
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models

In [5]:
#train image
print(mnist.train.images)
print(len(mnist.train.images))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
55000


In [6]:
#train label
print(mnist.train.labels)
print(len(mnist.train.labels))

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
55000


In [12]:
#첫번째 레이블 데이터 확인
print(len(mnist.train.labels[0]))
print(mnist.train.labels[0])

10
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [14]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#MNIST 데이터 읽어들이기
mnist=input_data.read_data_sets("mnist/", one_hot=True)

pixels=28*28 
nums=10 # 0-9사이의 카테고리

#placeholder 정의하기
x=tf.placeholder(tf.float32,shape=(None, pixels), name="x") #image data
y_=tf.placeholder(tf.float32,shape=(None, nums), name="y_") #result label

#가중치와 바이어스를 초기화하는 함수
def weight_variable(name, shape):
    W_init=tf.truncated_normal(shape,stddev=0.1)
    W=tf.Variable(W_init, name="W_"+name)
    return W

def bias_variable(name, size):
    b_init=tf.constant(0,1,shape=[size])
    b=tf.Variable(b_init, name="b_"+name)
    return b

#합성곱 계층을 만드는 함수
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

#최대 풀링층
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

#합성곱층1
with tf.name_scope('conv1') as scope:
    W_conv1=weight_variable('conv1',[5,5,1,32]) # 5*5 filter, 입력 채녈 1 (흑백), 출력 채녈 32
    b_conv1=bias_variable('conv1',32)
    x_image=tf.reshape(x,[-1,28,28,1])
    h_conv1=tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
    
#풀링층1
with tf.name_scope('pool1') as scope:
    h_pool1=max_pool(h_conv1)
    
#합성곱층2
with tf.name_scope('conv2') as scope:
    W_conv2=weight_variable('conv2', [5,5,32,64])
    b_conv2=bias_variable('conv2', 64)
    h_conv2=tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    
#풀링층2
with tf.name_scope('pool2') as scope:
    h_pool2=max_pool(h_conv2)
    
#fully connected
with tf.name_scope('fully_connected') as scope:
    n=7*7*64 #2*2 pooling 2번 = 28/2/2 -> 7*7
    W_fc=weight_variable('fc', [n,1024])
    b_fc=bias_variable('fc', 1024)
    h_pool2_flat=tf.reshape(h_pool2, [-1,n])
    h_fc=tf.nn.relu(tf.matmul(h_pool2_flat, W_fc)+b_fc)
    
#dropout
with tf.name_scope('dropout') as scope:
    keep_prob=tf.placeholder(tf.float32)
    h_fc_drop=tf.nn.dropout(h_fc, keep_prob)
    
#출력층
with tf.name_scope('readout') as scope:
    W_fc2=weight_variable('fc2', [1024,10])
    b_fc2=bias_variable('fc2',10)
    y_conv=tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2)+b_fc2)
    
#모델 학습시키기
with tf.name_scope('loss') as scope:
    cross_entoropy=-tf.reduce_sum(y_*tf.log(y_conv))
with tf.name_scope('training') as scope:
    optimizer=tf.train.AdamOptimizer(1e-4) # 확률적 SGD
    train_step=optimizer.minimize(cross_entoropy)
    
#모델 평가하기
with tf.name_scope('predict') as scope:
    predict_step=tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy_step=tf.reduce_mean(tf.cast(predict_step, tf.float32))
    
#feed_dict 설정하기
def set_feed(images,labels, prob):
    return {x:images, y_:labels, keep_prob: prob}

#세션 시작하기
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #Tensorboard 
    tw=tf.summary.FileWriter("log_dir",graph=sess.graph)
    #test 전용 피드
    test_fd=set_feed(mnist.test.images, mnist.test.labels,1)
    #학습시작
    for step in range(10000):
        batch=mnist.train.next_batch(50) #50개 이미지 10000번 학습
        fd=set_feed(batch[0], batch[1], 0.5)
        _, loss=sess.run([train_step, cross_entoropy], feed_dict=fd)
        if step % 100 ==0:
            acc=sess.run(accuracy_step,feed_dict=test_fd)
            print("step=", step, " loss=", loss, " acc=", acc)
            
    #최종결과출력
    acc=sess.run(accuracy_step, feed_dict=test_fd)
    print("정답률=",acc)
        

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
step= 0  loss= 464.98083  acc= 0.0993
step= 100  loss= 55.826004  acc= 0.8725
step= 200  loss= 34.281128  acc= 0.9116
step= 300  loss= 11.065873  acc= 0.9337
step= 400  loss= 12.939747  acc= 0.9455
step= 500  loss= 8.577711  acc= 0.9511
step= 600  loss= 25.6171  acc= 0.9557
step= 700  loss= 3.3858705  acc= 0.9613
step= 800  loss= 8.006658  acc= 0.9634
step= 900  loss= 6.199937  acc= 0.9675
step= 1000  loss= 17.926603  acc= 0.9705
step= 1100  loss= 4.4213977  acc= 0.9718
step= 1200  loss= 17.938478  acc= 0.9733
step= 1300  loss= 10.216834  acc= 0.9736
step= 1400  loss= 4.534616  acc= 0.9756
step= 1500  loss= 3.47437  acc= 0.9767
step= 1600  loss= 16.357033  acc= 0.9781
step= 1700  loss= 0.88176775  acc= 0.9773
step= 1800  loss= 6.7876563  acc= 0.9792
step= 1900  loss= 1.7074162  acc= 0.9802
step= 2000  loss= 6.02387